In [167]:
import numpy as np
import pandas as pd
from stempel import StempelStemmer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import copy
from sklearn.feature_extraction.text import CountVectorizer

In [302]:
def load_file(filepath):
    """
    Wczytuje plik tekstowy i zwraca jego zawartość jako string
    """
    with open(filepath, encoding="utf-8") as f:
        text = f.read()
    return text

def create_description_column(filepath, posting_delimiter='********'):
    """
    Potrzebne tylko kiedy wczytujemy dane z pliku tekstowego, w przypadku wczytywania z csv zamiast outputu tej funkcji podajemy df['description']
    """
    text = load_file(filepath)
    
    text_transformed = text.replace('\n', ' ').replace('\r', '')
    opisy_ofert = text_transformed.split(posting_delimiter)
    return opisy_ofert

def preprocess_text(description_column, stopwords, stemmer):
    """
    Usuwanie stopwordsów oraz stemming
    """
    
    opisy_ofert_bez_stopwords = []
    for opis in description_column:
        opis_bez_stopwords = opis
        for stopword in stopwords:
            opis_bez_stopwords = re.sub(r"\b%s\b" %stopword, '', opis_bez_stopwords)
        opisy_ofert_bez_stopwords.append(opis_bez_stopwords)

    opisy_po_stemmingu = []
    for opis in opisy_ofert_bez_stopwords:
        opisy_po_stemmingu.append([stemmer.stem(word.lower()) for word in opis.split()])

    return opisy_po_stemmingu


def create_tfidf_frame(opisy_ofert):
    """
    Tworzy ramkę danych ze słowami jako kolumnami i wartościami tfidf
    """

    vectorizer = TfidfVectorizer()
    cechy_tfidf = vectorizer.fit_transform(opisy_ofert)

    tfidf = cechy_tfidf.toarray()
    cechy_df = pd.DataFrame(tfidf, columns=vectorizer.get_feature_names_out())

    # Wyświetlanie ramki danych
    return cechy_df


def create_tfidf_columns(df_tfidf):
    tfidf_cols = pd.DataFrame([df_tfidf.iloc[:,:-1].sum(axis=1), df_tfidf.iloc[:,:-1].mean(axis=1)]).T
    tfidf_cols.columns=['tfidf_sum', 'tfidf_mean']
    return tfidf_cols

def create_emotions_columns(opisy_ofert, grouped_emotions):
    # Tworzenie wektora cech
    vectorizer = CountVectorizer()
    cechy = vectorizer.fit_transform(opisy_ofert)

    # Konwersja wektora cech do ramki danych
    df_emotions = pd.DataFrame(cechy.toarray(), columns=vectorizer.get_feature_names_out())

    for column in  list(df_emotions.columns):
        if column in list(grouped_emotions.index):
            df_emotions[column] *= grouped_emotions.loc[column,'emotions']

    emotion_cols = pd.DataFrame([df_emotions.sum(axis=1), df_emotions.mean(axis=1)]).T
    emotion_cols.columns=['emotions_sum', 'emotions_mean']
    return emotion_cols

def create_text_based_columns(df, text_column):
    df['text_length'] = df[text_column].apply(len)

    #liczba dużych liter
    df['capital_letters_count'] = df[text_column].apply(lambda x: sum(1 for c in x if c.isupper()))

    #zliczenia liczb
    df['numbers_count'] = df[text_column].apply(lambda x: sum(1 for c in x if c.isdigit()))

    #zliczenia znaków
    df['question_marks_count'] = df[text_column].apply(lambda x: x.count('!'))

    #zliczenia znaków walutowych
    df['currency_signs_count'] = df[text_column].apply(lambda x: x.count('$') + x.count('€') + x.count('£') + len(re.findall(r'\bzł\b|zł\b', x, re.IGNORECASE)))

        #liczba słów całych wielka litera
    df['capital_words_count'] = df[text_column].apply(lambda x: sum(1 for word in x.split() if word.isupper()))

    #możliwy adres email
    df['possible_email'] = df[text_column].apply(lambda x: ','.join([word for word in x.split() if '@' in word]))

    #możliwy adres rzeczywisty
    df['possible_address'] = df[text_column].apply(lambda x: ', '.join(re.findall(r'.{0,10}ul\..{0,10}', x)))

    #niepolskie słowa
    df['non_polish_char_count'] = df[text_column].apply(lambda x: sum(1 for char in ''.join(x) if char.strip() and (not char.isalpha() or char.lower() not in 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzźż')))

    #możliwy numer telefonu
    df['possible_phone_numbers'] = df[text_column].apply(lambda x: re.findall(r'\d+(?:[-\s]?\d+)+', x))
    df['possible_phone_numbers'] = df['possible_phone_numbers'].apply(lambda x: ', '.join(number for number in x if sum(1 for c in number if c.isdigit()) >= 9 and sum(1 for c in number if c.isdigit()) < 13))
    return df

def create_keywords_counter_column(df, keywords, ispreprocessed=False):
    # final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
    # final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
    # final_df = create_text_based_columns(final_df, 'description')


    # all_keywords = ["dowód" , "zawsze" , "nigdy" , "pesel" , "kryptowaluty" ,  "nic" , "wszystko" , "konto bankowe"]
    if ispreprocessed:
        text_column = 'preprocessed_description'
    else:  
        text_column = 'description'
    keywords_counter_col = [0]* len(df[text_column])

    for i in range(len(df['description'])):
        for keyword in keywords:
            keywords_counter_col[i] += len(re.findall(keyword, df[text_column][i].lower()))

    df['keywords_counter'] = keywords_counter_col
    return df

In [303]:
with open('data\stop_words_polish.txt', encoding="utf-8") as f:
    stopwords = f.readlines()

for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].replace('\n', '')

In [241]:
stemmer = StempelStemmer.polimorf()

Loading: 100%|██████████| 11368252/11368252 [00:16<00:00, 679664.90bytes/s] 


In [304]:
opisy_pozytywne_desc = create_description_column('data\FakeJobHunter - pozytywne.txt',"Opis ogłoszenia")
opisy_negatywne_desc = create_description_column('data\FakeJobHunter - negatywne.txt')

opisy_pozytywne = preprocess_text(opisy_pozytywne_desc, stopwords, stemmer)
opisy_negatywne = preprocess_text(opisy_negatywne_desc, stopwords, stemmer)

In [305]:
opisy_pozytywne_full = []
opisy_negatywne_full = []

for opis in opisy_pozytywne:
    try:
        opisy_pozytywne_full.append(' '.join(opis))
    except:
        print(opis)
        opisy_pozytywne_full.append('INVALID_DATA')
        continue

for opis in opisy_negatywne:
    try:
        opisy_negatywne_full.append(' '.join(opis))
    except:
        print(opis)
        opisy_negatywne_full.append('INVALID_DATA')
        continue

['nativus', 'dzień', 'doradca:', 'mieć', 'bezpośredni', 'kontakt', 'nasze', 'klient', 'zapewniać', 'kompleksowy', 'obsługę,', 'być', 'profesjonalny', 'budować', 'przyjazny', 'relacja', 'klientami,', 'pracować', 'swój', 'premię,', 'prowadzić', 'aktywny', 'sprzedaż', 'produkt', 'usługi', 'bankowy', 'ubezpieczeniowych,', 'przeprowadzać', 'transakcja', 'gotówkowy', 'bezgotówkowe.', 'to', 'stanowisko', 'twoje,', ':', 'interesować', 'branża', 'finansowy', '–', 'chcieć', 'nenen', 'wiedza', 'rozwijać,', 'lubić', 'ludzi.', 'n', 'bać', 'kontakt', 'klient', 'być', 'otwarty/', 'potrzeby,', 'podchodzić', 'entuzjastycznić', 'praca', 'chcieć', 'mść', 'satysfakcja', 'osiągać', 'celów,', 'mieć', 'wykształcić', 'średnia', 'wyższe,', 'mieć', 'doświadczyć', 'praca', 'związać', 'finansami,', 'sprzedaż', 'obsługa', 'klient', '(mila', 'widziane).', 'dołączać', 'otrzymasz:', 'umowa', 'pracę,', 'stały', 'wynagrodzić', 'premię,', 'atrakcyjny', 'pakiet', 'benefit', 'karta', 'multisport,', 'prywatny', 'opieka', '

In [306]:
len(opisy_pozytywne_full), len(opisy_negatywne_full)

(14, 25)

In [307]:
etykiety = [1] * len(opisy_pozytywne_full) + [0] * len(opisy_negatywne_full)
opisy_ofert_full = opisy_pozytywne_full + opisy_negatywne_full
opisy_ofert_desc = opisy_pozytywne_desc + opisy_negatywne_desc



df_desc_preprocessed = pd.DataFrame([opisy_ofert_desc, opisy_ofert_full]).T
df_desc_preprocessed.columns = ['description', 'preprocessed_description']
df_desc_preprocessed['label'] = etykiety
df_desc_preprocessed['index'] = df_desc_preprocessed.index
df_desc_preprocessed = df_desc_preprocessed[df_desc_preprocessed['preprocessed_description'] != 'INVALID_DATA'].reset_index(drop=True)
df_desc_preprocessed


,description,preprocessed_description,label,index
0,,,1,0
1,SAMODZIELNA KSIĘGOWA Twój zakres obowiązków ...,samodzielny księgowy twój zakres obowiązek ksi...,1,1
2,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,2
3,"Zatrudnimy doracę, doradczynię ds. kontaktu z...","zatrudnić doracę, doradczyni ds. kontakt klien...",1,3
4,Jeśli: w pracy nie lubisz rutyny szukasz sta...,jeśli: praca lubić rutyna szukać stały dodatko...,1,4
5,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,5
6,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,6
7,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,7
8,"Szukam ambitnych osób do zespołu, które nie b...","szukać ambitny osoba zespołu, boja nowy wyzwań...",1,8
9,LUBISZ DORADZAĆ INNYM? RÓB TO Z NAMI! Doradza...,lubić doradzać innym? robić to ezyt nami! dora...,1,9


In [308]:

df_tfidf = create_tfidf_frame(df_desc_preprocessed['preprocessed_description'])
tfidf_cols = create_tfidf_columns(df_tfidf)
tfidf_cols


,tfidf_sum,tfidf_mean
0,0.000000,0.000000
1,10.145443,0.006282
2,11.200993,0.006936
3,3.341539,0.002069
4,5.797564,0.003590
5,9.428065,0.005838
6,9.375801,0.005805
7,9.420874,0.005833
8,6.901856,0.004274
9,9.819691,0.006080


In [309]:
grouped_emotions = pd.read_csv('data\grouped_emotions_dictionary.csv')
grouped_emotions.set_index('word', inplace=True)
grouped_emotions

,emotions
word,
$,0
$L,0
$T,0
"1,3-butadien",0
1/2,0
...,...
′,0
″,0
€,0


In [310]:
# UWAGA - długo się wykonuje
emotion_cols = create_emotions_columns(df_desc_preprocessed['preprocessed_description'], grouped_emotions)
emotion_cols

,emotions_sum,emotions_mean
0,0.0,0.000000
1,100.0,0.061881
2,117.0,0.072401
3,10.0,0.006188
4,34.0,0.021040
5,82.0,0.050743
6,71.0,0.043936
7,77.0,0.047649
8,48.0,0.029703
9,86.0,0.053218


In [311]:
final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
final_df = create_text_based_columns(final_df, 'description')


all_keywords = ["dowód" , "zawsze" , "nigdy" , "pesel" , "kryptowaluty" ,  "nic" , "wszystko" , "konto", "niepowtarzalna",
    "natychmiastowe",
    "niesamowite",
    "ekstra",
    "gwarantowane",
    "ekscytujące",
    "najlepsze",
    "super",
    "natychmiast",
    "wyjątkowe"]
#all_keywords_stemmed = [stemmer.stem(word) for word in all_keywords]

final_df = create_keywords_counter_column(final_df, all_keywords)
final_df

,description,preprocessed_description,label,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,capital_words_count,possible_email,possible_address,non_polish_char_count,possible_phone_numbers,keywords_counter
0,,,1,0,0.000000,0.000000,0.0,0.000000,0,0,0,0,0,0,,,0,,0
1,SAMODZIELNA KSIĘGOWA Twój zakres obowiązków ...,samodzielny księgowy twój zakres obowiązek ksi...,1,1,10.145443,0.006282,100.0,0.061881,1657,51,4,0,0,5,,,31,,1
2,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,2,11.200993,0.006936,117.0,0.072401,1744,40,15,1,0,3,,,49,,1
3,"Zatrudnimy doracę, doradczynię ds. kontaktu z...","zatrudnić doracę, doradczyni ds. kontakt klien...",1,3,3.341539,0.002069,10.0,0.006188,128,4,0,0,0,0,,,5,,0
4,Jeśli: w pracy nie lubisz rutyny szukasz sta...,jeśli: praca lubić rutyna szukać stały dodatko...,1,4,5.797564,0.003590,34.0,0.021040,416,3,0,0,0,0,,,2,,0
5,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,5,9.428065,0.005838,82.0,0.050743,1316,31,15,1,0,0,,,34,,1
6,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,6,9.375801,0.005805,71.0,0.043936,1075,28,17,1,1,2,,,40,,1
7,Manpower (Agencja zatrudnienia nr 412) to glo...,manpower (agencja zatrudnienie wumer 412) glob...,1,7,9.420874,0.005833,77.0,0.047649,1082,26,22,1,1,1,,,41,,1
8,"Szukam ambitnych osób do zespołu, które nie b...","szukać ambitny osoba zespołu, boja nowy wyzwań...",1,8,6.901856,0.004274,48.0,0.029703,579,13,1,0,0,0,,,14,,0
9,LUBISZ DORADZAĆ INNYM? RÓB TO Z NAMI! Doradza...,lubić doradzać innym? robić to ezyt nami! dora...,1,9,9.819691,0.006080,86.0,0.053218,1217,77,0,2,0,14,,,42,,0


In [312]:
final_df.to_csv("data\example_data_v1.csv")

## Real Data

In [277]:
df = pd.read_csv("data\olx.csv")

In [285]:
#preprocess_text(df['description'], stopwords, stemmer)
opisy = preprocess_text(df['description'].replace('\n', '').replace('\r', ''), stopwords, stemmer)

In [286]:
opisy_full = []

for opis in opisy:
    try:
        opisy_full.append(' '.join(opis))
    except:
        print(opis)
        opisy_full.append('INVALID_DATA')
        continue

['lektor', 'język', 'angielskiegopraca', 'szkoła', 'językowy', 'radomiujeśli:ciekawy', 'ty', 'praca', 'ludźmi,', 'szczególność', 'dziecko', 'wykorzystywać', 'nowy', 'metoda', 'technologia', 'nauczanie', 'języka,chcieć', 'uczyć', 'dziecko', 'efektywny', 'metodą,', 'dzięki', 'zajęcie', 'przygotowywać', 'mgnienie', 'oka,', 'mieć', 'gotowy', 'scenariusz', 'lekcje', 'przeszkolenie,mieć', 'umiejętność', 'interpersonalny', '–', 'lubić', 'praca', 'człowiek', 'dostarczać', 'ty', 'pokład', 'energii,potrafić', 'motywować', 'swój', 'słuchacz', 'siebie,rozwój', 'osobisty', 'doskonalić', None, 'ty', 'integralny', 'część', 'życia,brać', 'pełny', 'odpowiedzialność', 'podjąć', 'zadanie', 'ty', 'polegać,być', 'komunikatywny', 'tryskać', 'energiąkochać', 'uśmiechaćkonstruktywny', 'krytyka', 'odbierać', 'wdzięcznością,', 'wiesz,', 'przejawić', 'życzliwość', 'osób,', 'odnajdywać', 'świat', 'język', 'obcych,…', 'być', 'osobą,', 'szukamy!być', 'szkoła', 'językowy', 'leonarda', 'school', 'dziecko', 'młodzieży

In [296]:
df_desc_preprocessed = pd.DataFrame([list(df['description']), opisy_full]).T
df_desc_preprocessed.columns = ['description', 'preprocessed_description']
df_desc_preprocessed['index'] = df_desc_preprocessed.index
df_desc_preprocessed = df_desc_preprocessed[df_desc_preprocessed['preprocessed_description'] != 'INVALID_DATA'].reset_index(drop=True)
df_desc_preprocessed

,description,preprocessed_description,index
0,Najlepsze oferty pracy w budownictwie\r\nTel: ...,dobry oferta praca budownictwo tel: 88*******7...,0
1,Na tym stanowisku będziesz odpowiedzialny za:m...,nativus stanowisko być odpowiedzialny :montaż ...,1
2,"Akademia Nauki zatrudni NAUCZYCIELI, PSYCHOLOG...","akademia nauka zatrudnić nauczycieli, psycholo...",2
3,Dołącz do nas! Do szybko rosnącej firmy w obsz...,dołączyć ! dojciec szybki róść firma obszar en...,3
4,Nowo odnowione Centrum stomatologii w Krakowie...,nowy odnowić centrum stomatologia krak zatrudn...,4
...,...,...,...
2419,firmy:Ponad 3100 sklepów w 1100 miejscowości ...,firmy:ponad 3100 sklep 1100 miejscowość 16 cen...,2486
2420,Witam\r\n\r\nDo zaoferowania mam pracę jako ki...,witać dojciec zaoferować praca kierowca pojazd...,2487
2421,Agencja Ochrony Osób i Mienia zatrudni osobę n...,agencja ochrona osoba mienie zatrudnić osoba s...,2488
2422,Firma z wieloletnim doświadczeniem zajmująca s...,firma wieloletni doświadczyć zajmujący sprzeda...,2489


In [297]:
df_tfidf = create_tfidf_frame(df_desc_preprocessed['preprocessed_description'])
tfidf_cols = create_tfidf_columns(df_tfidf)
tfidf_cols

,tfidf_sum,tfidf_mean
0,8.381325,0.000282
1,8.790453,0.000296
2,9.400964,0.000316
3,9.881147,0.000333
4,8.143853,0.000274
...,...,...
2419,9.479701,0.000319
2420,4.645192,0.000156
2421,7.896931,0.000266
2422,5.263060,0.000177


In [298]:
grouped_emotions = pd.read_csv('data\grouped_emotions_dictionary.csv')
grouped_emotions.set_index('word', inplace=True)
grouped_emotions

,emotions
word,
$,0
$L,0
$T,0
"1,3-butadien",0
1/2,0
...,...
′,0
″,0
€,0


In [299]:
emotion_cols = create_emotions_columns(df_desc_preprocessed['preprocessed_description'], grouped_emotions)
emotion_cols

,emotions_sum,emotions_mean
0,76.0,0.002558
1,89.0,0.002995
2,90.0,0.003029
3,93.0,0.003130
4,60.0,0.002019
...,...,...
2419,96.0,0.003231
2420,18.0,0.000606
2421,54.0,0.001817
2422,24.0,0.000808


In [300]:
final_df = df_desc_preprocessed.merge(tfidf_cols, left_index=True, right_index=True)
final_df = final_df.merge(emotion_cols, left_index=True, right_index=True)
final_df = create_text_based_columns(final_df, 'description')


all_keywords = ["dowód" , "zawsze" , "nigdy" , "pesel" , "kryptowaluty" ,  "nic" , "wszystko" , "konto", "niepowtarzalna",
    "natychmiastowe",
    "niesamowite",
    "ekstra",
    "gwarantowane",
    "ekscytujące",
    "najlepsze",
    "super",
    "natychmiast",
    "wyjątkowe"]
#all_keywords_stemmed = [stemmer.stem(word) for word in all_keywords]

final_df = create_keywords_counter_column(final_df, all_keywords)
final_df

,description,preprocessed_description,index,tfidf_sum,tfidf_mean,emotions_sum,emotions_mean,text_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,capital_words_count,possible_email,possible_address,non_polish_char_count,possible_phone_numbers,keywords_counter
0,Najlepsze oferty pracy w budownictwie\r\nTel: ...,dobry oferta praca budownictwo tel: 88*******7...,0,8.381325,0.000282,76.0,0.002558,1165,29,16,0,1,0,,,66,,2
1,Na tym stanowisku będziesz odpowiedzialny za:m...,nativus stanowisko być odpowiedzialny :montaż ...,1,8.790453,0.000296,89.0,0.002995,1342,18,22,0,0,1,,,47,"48 14 610 01 40, 48 14 610 01 45",1
2,"Akademia Nauki zatrudni NAUCZYCIELI, PSYCHOLOG...","akademia nauka zatrudnić nauczycieli, psycholo...",2,9.400964,0.000316,90.0,0.003029,1276,141,27,0,1,17,,,72,,0
3,Dołącz do nas! Do szybko rosnącej firmy w obsz...,dołączyć ! dojciec szybki róść firma obszar en...,3,9.881147,0.000333,93.0,0.003130,1361,27,0,4,0,2,,,34,,2
4,Nowo odnowione Centrum stomatologii w Krakowie...,nowy odnowić centrum stomatologia krak zatrudn...,4,8.143853,0.000274,60.0,0.002019,892,20,19,0,0,0,,,42,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2419,firmy:Ponad 3100 sklepów w 1100 miejscowości ...,firmy:ponad 3100 sklep 1100 miejscowość 16 cen...,2486,9.479701,0.000319,96.0,0.003231,1299,32,24,1,2,2,,,57,,0
2420,Witam\r\n\r\nDo zaoferowania mam pracę jako ki...,witać dojciec zaoferować praca kierowca pojazd...,2487,4.645192,0.000156,18.0,0.000606,239,11,4,0,0,1,,,6,,0
2421,Agencja Ochrony Osób i Mienia zatrudni osobę n...,agencja ochrona osoba mienie zatrudnić osoba s...,2488,7.896931,0.000266,54.0,0.001817,851,26,4,0,0,2,,,20,,0
2422,Firma z wieloletnim doświadczeniem zajmująca s...,firma wieloletni doświadczyć zajmujący sprzeda...,2489,5.263060,0.000177,24.0,0.000808,358,5,4,2,0,0,,,11,,1


In [301]:
final_df.to_csv('data\df_final_v1.csv')